In [1]:
import os
import sys
sys.path.append("../")
from datatools.analyzer import *

from datatools.maneger import DataManager
import pandas as pd

/home/yamada/.local/share/virtualenvs/twitchAI-AEfI_JA6/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def clean_text(text):
    text_ = neologdn.normalize(text)
    text_ = re.sub(r'\([^\)]*\)', "", text_)
    # text_ = re.sub(r'\d+', "0", text_)
    text_  = "".join( [m.normalized_form() if m.part_of_speech()[0]=="名詞" else m.surface() for m in tokenizer_obj.tokenize(text_, tmode)] )
    text_ = text_.replace("??", "?")
    return text_


In [3]:
def load_utt_ntt():
    ntt_path = "../../corpus/NTT/"
    utt_list = []

    for file_ in os.listdir(ntt_path):
        if not "json" in file_:
            continue 
        with open(ntt_path+file_, "r",  encoding="utf-8") as f:
            convs = json.load(f)
            for did in convs["convs"]:
                dids = list( did.keys() )[0]
                conv = did[dids]
                # conv = did[dids][3::3]
                utt_list.append( [ clean_text(utt)  for utt in conv])
    
    print(len(utt_list))
    return utt_list

In [4]:
ntt_utt = load_utt_ntt()

25015


In [5]:
def format_seq(conv):
    if len(conv)%2 != 0:
        conv = conv[:-1]
    return conv

def make_pair(convs):
    pair_list = []

    for conv in convs:
        # pair_list += np.array(format_seq(conv)).reshape(-1, 2).tolist()
        for i in range(len(conv)-2):
            pair_list.append(conv[i:i+3])
    
    return pair_list

In [6]:
pair_list = make_pair(ntt_utt)

In [7]:
len(pair_list)

91747

In [11]:
pair_list[:10]

[['こんにちは。お元気ですか?', 'はい、元気です。広告代理店での仕事が忙しいですが。', 'お疲れ様です。私は介護福祉士をしています。'],
 ['はい、元気です。広告代理店での仕事が忙しいですが。',
  'お疲れ様です。私は介護福祉士をしています。',
  '介護福祉士として働いていらっしゃるんですね。大変なお仕事ですよね。'],
 ['お疲れ様です。私は介護福祉士をしています。',
  '介護福祉士として働いていらっしゃるんですね。大変なお仕事ですよね。',
  'いえ。広告代理店も大変ですよね。因みに私は北海道の一軒家に住んでいます。'],
 ['介護福祉士として働いていらっしゃるんですね。大変なお仕事ですよね。',
  'いえ。広告代理店も大変ですよね。因みに私は北海道の一軒家に住んでいます。',
  '北海道ですか。御飯の美味しい所で羨ましいです。私は青森県出身で山の多い所が好きなので、山の近くに住んでいます'],
 ['いえ。広告代理店も大変ですよね。因みに私は北海道の一軒家に住んでいます。',
  '北海道ですか。御飯の美味しい所で羨ましいです。私は青森県出身で山の多い所が好きなので、山の近くに住んでいます',
  '北海道は、美味しい物多いですよ。青森なら、お近くでしたね。でも、私はいつか東京に住んでみたいんです。'],
 ['北海道ですか。御飯の美味しい所で羨ましいです。私は青森県出身で山の多い所が好きなので、山の近くに住んでいます',
  '北海道は、美味しい物多いですよ。青森なら、お近くでしたね。でも、私はいつか東京に住んでみたいんです。',
  '東京ですか。東京には華やかな場所が多いですよね'],
 ['北海道は、美味しい物多いですよ。青森なら、お近くでしたね。でも、私はいつか東京に住んでみたいんです。',
  '東京ですか。東京には華やかな場所が多いですよね',
  '田舎生まれの私にとって、とても魅力的な場所です。'],
 ['東京ですか。東京には華やかな場所が多いですよね',
  '田舎生まれの私にとって、とても魅力的な場所です。',
  '私にとっても東京は魅力的な場所ですね。お休みの日は何をされていますか?私は写真が趣味です。'],
 ['田舎生まれの私にとって、とても魅力的な場所です。',
  '私にとっても東京は魅力的

In [9]:
out_path = "../../corpus/novel_formated/"
corpus_name = "ntt_segment.tsv"
import csv

In [10]:
with open(out_path+corpus_name, "w") as f:
    writer = csv.writer(f, delimiter="\t")
    writer.writerows(pair_list)

In [59]:
def plain_convs(pair_list):
    plain_pair_list = []

    for pair in tqdm(pair_list):
        new_pair = []
        
        # 直前の発話
        # divp = [ s.orth_ for s in (nlp(pair[1]).sents)]
        divp = pair[1].split("。")
        if len(divp) <= 1:
            new_pair.append(pair[1])
        else:
            utt = divp[-2]
            new_pair.append(divp[-2])
        
        # 生成対象の発話
        # divp = pair[2].split("。")
        divp = [ s.orth_ for s in (nlp(pair[2]).sents)]
        if len(divp) <= 1:
            new_pair.append(pair[2])
        else:
            new_pair.append(divp[0])
        plain_pair_list.append(new_pair)

    return plain_pair_list

In [61]:
plain_pair_list = plain_convs(pair_list[:50000])
plain_pair_list

100%|██████████| 50000/50000 [11:10<00:00, 74.55it/s]


[['広告代理店での仕事が忙しいですが', 'お疲れ様です。'],
 ['私は介護福祉士をしています', '介護福祉士として働いていらっしゃるんですね。'],
 ['大変なお仕事ですよね', 'いえ。'],
 ['因みに私は北海道の一軒家に住んでいます', '北海道ですか。'],
 ['御飯の美味しい所で羨ましいです', '北海道は、美味しい物多いですよ。'],
 ['でも、私はいつか東京に住んでみたいんです', '東京ですか。'],
 ['東京ですか', '田舎生まれの私にとって、とても魅力的な場所です。'],
 ['田舎生まれの私にとって、とても魅力的な場所です', '私にとっても東京は魅力的な場所ですね。'],
 ['お休みの日は何をされていますか?私は写真が趣味です', '私は彼氏いない歴も長くて、休みの日は一人でのんびりしています。'],
 ['写真、いいですね', '一人でのんびりと過ごす時間も大切で、贅沢な物ですよ。'],
 ['私もよく家でのんびり過ごしています', '一人の時間も大切ですよね。'],
 ['仕事も忙しいので、ゆっくりしたいですよね', 'そうですね。'],
 ['はい、元気にしていました', '私も元気にしていました。'],
 ['休みの日には趣味の写真を楽しんでいます', '写真ですか、いいですね。'],
 ['私は海の近くに住んでいるので、よく散歩をしています', '海の近くにお住まいなんですね。'],
 ['私は山のそばに住んでいます', 'そうなんですね。'],
 ['山も好きなのですが、沖縄に以前住んでいてどうしても海の近くに住みたくて', '生まれ育った場所に似た場所はなんだか心が落ち着きますよね。'],
 ['私も青森出身なのでお気持ちわかります', 'そうですよね。'],
 ['そうですよね', '自然は心を落ち着かせてくれますよね。'],
 ['私も今度海辺の写真を撮りに行きたいです', 'はい、海もさまざまな表情を見せますのでとても良いと思います。'],
 ['はい、海もさまざまな表情を見せますのでとても良いと思います', 'また御勧めの場所があったらぜひ教えてくださいね。'],
 ['また御勧めの場所があったらぜひ教えてくださいね', 'はい。'],
 ['はい', '私はダンスにはなかなか縁がなくて、めっきり

In [62]:
corpus_name = "ntt_segment_plain2.tsv"
with open(out_path+corpus_name, "w") as f:
    writer = csv.writer(f, delimiter="\t")
    writer.writerows(plain_pair_list)